In [148]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, TransformChain, SequentialChain
from tqdm import trange
import json
from pprint import pprint
import re
# import python-dotenv and use it to set OPENAI_API_KEY as an environment variable
from dotenv import load_dotenv
load_dotenv()

from stegosaurus.acrostic import acrostic_generator_prompt, acrostic_evaluator_prompt, multi_acrostic_generator_prompt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
def clean_output(s):
    s = s.strip()
    # if there is more than one sentence, keep only the first one
    # (this is a hack to deal with the fact that the prompt is not always followed)
    # cut anything after a sentence ending punctuation, but keep the punctuation itself
    if '.' in s:
        s = s[:s.index('.')+1]
    elif '?' in s:
        s = s[:s.index('?')+1]
    elif '!' in s:
        s = s[:s.index('!')+1]
    return s
    
    # reformat the outputs to be a single string containing a numbered list of sentences, one per line
def outputs_to_multiple_choice(outputs):
    output_string = ""
    for i, output in enumerate(outputs):
        output_string += f"{i+1}. {output['cleaned_new_sentence']}\n"
    output_string += f'{len(outputs)+1}. None of these look good. AGA, try again.'
    return output_string

def get_selection_from_evaluator_output(output):
    try:
        return int(json.loads(output['evaluator_output'])['selected_option'])
    except:
        return output

In [100]:
def generate_options(
    next_sentence_chain, 
    original_text, 
    acrostic_phrase, 
    rewritten_text, 
    current_starting_letter,
    n_attempts:int=5,
    ):
    outputs = []
    inputs = dict(
        original_text=original_text,
        acrostic_phrase=acrostic_phrase,
        rewritten_text=rewritten_text,
        current_starting_letter=current_starting_letter,)
    for i in trange(n_attempts):
        output = next_sentence_chain(
            inputs=inputs
            )
        output['cleaned_new_sentence'] = clean_output(output['new_sentence'])
        outputs.append(output)
        
    options = outputs_to_multiple_choice(outputs)
    n_options = len(outputs)
    return outputs, options, n_options


def evaluate_options(
    acrostic_evaluator_chain,
    original_text,
    acrostic_phrase,
    rewritten_text,
    current_starting_letter,
    options,
    ):
    inputs = {
            'original_text': original_text,
            'acrostic_phrase': acrostic_phrase,
            'rewritten_text': rewritten_text,
            'current_starting_letter': current_starting_letter,
            'options': options
        }
    evaluator_output = acrostic_evaluator_chain(inputs=inputs)
    evaluator_output['n_options'] = n_options
    evaluator_output['evaluator_selection'] = get_selection_from_evaluator_output(evaluator_output)
    return evaluator_output

In [113]:
def gen_and_evaluate(
    next_sentence_chain, 
    acrostic_evaluator_chain,
    original_text,
    acrostic_phrase,
    rewritten_text,
    current_starting_letter,
    n_attempts:int=5,
    ):
    outputs, options, n_options = generate_options(
        next_sentence_chain=next_sentence_chain, 
        original_text=original_text,
        acrostic_phrase=acrostic_phrase, 
        rewritten_text=rewritten_text,
        current_starting_letter=current_starting_letter,
        n_attempts=n_attempts,
        )
    evaluator_output = evaluate_options(
        acrostic_evaluator_chain=acrostic_evaluator_chain,
        original_text=original_text,
        acrostic_phrase=acrostic_phrase,
        rewritten_text=rewritten_text,
        current_starting_letter=current_starting_letter,
        options=options,
        )
    return outputs, options, n_options, evaluator_output

In [2]:
chatgpt = OpenAI(
    model_name='gpt-3.5-turbo',
    temperature=1,
    )
next_sentence_chain = LLMChain(
    llm=chatgpt, 
    prompt=acrostic_generator_prompt,
    output_key='new_sentence',
    )

original_text = "On Earth, volcanoes are most often found where tectonic plates are diverging or converging, and most are found underwater. For example, a mid-ocean ridge, such as the Mid-Atlantic Ridge, has volcanoes caused by divergent tectonic plates whereas the Pacific Ring of Fire has volcanoes caused by convergent tectonic plates. Volcanoes can also form where there is stretching and thinning of the crust's plates, such as in the East African Rift and the Wells Gray-Clearwater volcanic field and Rio Grande rift in North America. Volcanism away from plate boundaries has been postulated to arise from upwelling diapirs from the core–mantle boundary, 3,000 kilometers (1,900 mi) deep in the Earth. This results in hotspot volcanism, of which the Hawaiian hotspot is an example. Volcanoes are usually not created where two tectonic plates slide past one another."
acrostic_phrase = "FOOBAR"


acrostic_letter_index = 0
acrostic_letters = [letter for letter in acrostic_phrase]
current_starting_letter = acrostic_letters[acrostic_letter_index]
rewritten_text = ""


In [3]:

acrostic_evaluator_chain = LLMChain(
    llm=OpenAI(
        model_name='gpt-3.5-turbo',
        temperature=0.5,
        ), 
    prompt=acrostic_evaluator_prompt,
    output_key='evaluator_output',
    )



In [114]:
outputs, options, n_options, evaluator_output = gen_and_evaluate(
    next_sentence_chain, 
    acrostic_evaluator_chain,
    original_text,
    acrostic_phrase,
    rewritten_text,
    current_starting_letter,
    n_attempts=3,
    )

100%|██████████| 3/3 [00:03<00:00,  1.25s/it]


In [136]:
outputs, options, n_options, evaluator_output = gen_and_evaluate(
    next_sentence_chain, 
    acrostic_evaluator_chain,
    original_text,
    acrostic_phrase,
    rewritten_text,
    current_starting_letter,
    n_attempts=3,
    )

100%|██████████| 3/3 [00:04<00:00,  1.62s/it]


In [137]:
n_regenerations = 0

if accept_sentence(evaluator_output):
    rewritten_text += outputs[evaluator_output['evaluator_selection']-1]['cleaned_new_sentence'] + ' '
    acrostic_letter_index += 1
    current_starting_letter = acrostic_letters[acrostic_letter_index]
    print('Accepted sentence!')
else:
    print('Regenerating options...')
    n_regenerations += 1
    outputs, options, n_options, evaluator_output = gen_and_evaluate(
    next_sentence_chain, 
    acrostic_evaluator_chain,
    original_text,
    acrostic_phrase,
    rewritten_text,
    current_starting_letter,
    n_attempts=3,
    )


Accepted sentence!


In [135]:
pprint(rewritten_text)

('For the most part, volcanoes are located in areas where tectonic plates are '
 'either moving apart or coming together. Often these areas are found '
 'underwater, but there are also land-based examples such as the East African '
 'Rift. On the other hand, some of the most dramatic examples of volcanism '
 'occur at plate boundaries. But volcanoes can also form in areas where the '
 "crust's plates are stretched and thinned, like in the Wells Gray-Clearwater "
 'volcanic field and the Rio Grande rift in North America. ')


# try multi

In [199]:
from stegosaurus.acrostic.params import AcrosticGeneratorInputs, AcrosticGenerationRoundResult
from stegosaurus.acrostic.chains import full_chain

original_text = "On Earth, volcanoes are most often found where tectonic plates are diverging or converging, and most are found underwater. For example, a mid-ocean ridge, such as the Mid-Atlantic Ridge, has volcanoes caused by divergent tectonic plates whereas the Pacific Ring of Fire has volcanoes caused by convergent tectonic plates. Volcanoes can also form where there is stretching and thinning of the crust's plates, such as in the East African Rift and the Wells Gray-Clearwater volcanic field and Rio Grande rift in North America. Volcanism away from plate boundaries has been postulated to arise from upwelling diapirs from the core–mantle boundary, 3,000 kilometers (1,900 mi) deep in the Earth. This results in hotspot volcanism, of which the Hawaiian hotspot is an example. Volcanoes are usually not created where two tectonic plates slide past one another."
acrostic_phrase = "FOOBAR"


In [200]:
initial_inputs = AcrosticGeneratorInputs(
    original_text = """
I know a yellow labrador retriever named Lily.
She is a little over 2 years old, which is pretty much grown up.
She likes to play fetch a lot and it's a good way to tire her out.
She doesn't like to cuddle much, but as she gets older she likes it a bit more.
    """,
    acrostic_phrase = "LILY",
    rewritten_text = "",
    acrostic_letter_index = 0,
    n_attempts=1,
)
results = []
verbose = True
inputs = initial_inputs.copy()

In [201]:
current_round_number = 0
n_regenerations_for_current_round = 0

In [202]:
output = full_chain(inputs=inputs.dict())
current_round_result = AcrosticGenerationRoundResult(
    round_number=current_round_number,
    inputs=inputs.copy(),
    output=output,
)
results.append(current_round_result)

if output['next_action'] == 'accept sentence':
    
    selection_idx = output['evaluator_selection'] - 1
    accepted_sentence = output['generated_sentence_options_list'][selection_idx] + ' '
    inputs.rewritten_text += accepted_sentence
    print(f'Accepted sentence:\n{output["evaluator_selection"]}. {accepted_sentence}')
    inputs.acrostic_letter_index += 1
    n_regenerations_for_current_round = 0
else:
    print(f'{output["next_action"]}')
    n_regenerations_for_current_round += 1
current_round_number += 1

pprint(output['clean_options'])

inputs

Accepted sentence:
1. Lily is the name of the yellow labrador retriever that I am familiar with. 
('1. Lily is the name of the yellow labrador retriever that I am familiar '
 'with.\n'
 '2. None of these look good. AGA, try again.')



AcrosticGeneratorInputs

acrostic_phrase: LILY

original_text:
I know a yellow labrador retriever named Lily.
She is a little over 2 years old, which is pretty much grown up.
She likes to play fetch a lot and it's a good way to tire her out.
She doesn't like to cuddle much, but as she gets older she likes it a bit more.

acrostic_letter_index: 1
rewritten_text:
Lily is the name of the yellow labrador retriever that I am familiar with.

        